In [ ]:
import undetected_chromedriver.v2 as uc
from selenium.webdriver.common.by import By                                  ### Module
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from openpyxl import load_workbook
import time
import pandas as pd
import itertools 
import glob

In [ ]:
driver = uc.Chrome()  # loading undetectable chrome browser

driver.get('https://www.careerguide.com/career-options')  ## scraping careerguide website to get career names

In [ ]:
Catogery = []
State = []
Job_links = []
company_links = []
Company_Name = []
Job_location = []
Job_Title= []
Mode = []

df_bike_ = pd.DataFrame(columns=['Catogery','State','Job links', 'company links','Job Title','Job location','Company Name','Mode'])   # defining columns for excel file

In [ ]:
careerTitle = []  # list to store career title
careerTitleOpt= driver.find_elements(By.XPATH,"//*[@class='col-md-4']/ul/li")  ## scraping careerTitle from careerguide website

In [ ]:
for title in careerTitleOpt[1:11]:  ## only scraping for 10 job title
    careerTitle.append(title.text)

In [ ]:
driver.get('https://www.linkedin.com/')  ## getting linkedin website to scrape

In [ ]:
username = driver.find_element(By.ID,"session_key")               ## signing with username and password on linkedin
username.send_keys("chaudhary.s.shamshad07@gmail.com")           ## sending input text to input area
password = driver.find_element(By.ID,"session_password")
password.send_keys("Shamshad")                                   ##****your_password*****

In [ ]:
try:
    login_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME,"sign-in-form__submit-button"))   ## locator login button for linkedin
    )
    login_button.click()   # clicking login button
finally:
    print("cannot find Button")

In [ ]:
job_button=driver.find_element(By.XPATH,"(//*[@class='app-aware-link  global-nav__primary-link'])[2]") ## locator for job section
job_button.click()

In [ ]:
Locations = ['punjab','Delhi','Mumbai']  ## for now only scraping for three location
infos=[]
comp_links=[] ## list to store company links to scrape their info later on

In [ ]:
for Location in Locations:  ## double looping for each location with job careertitle
    for Title in careerTitle:
        url = "https://www.linkedin.com/jobs/search/?currentJobId=3348462787&geoId=106187582&keywords="+Title+"&location="+Location+"%2C%20India&refresh=true"
        
        driver.get(url)
        
        flag=1
        page_no = 25
        current_url = driver.current_url
            
        while(flag>0):
            ## locator for job info which as location, mode, title, company name separeted by newline
            info_div=driver.find_elements(By.XPATH,"//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']")

            ## locator for job links and company links
            job_Links = driver.find_elements(By.XPATH,"(//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']/div[1]/a)")
            comp_Links = driver.find_elements(By.XPATH,"(//*[@class='flex-grow-1 artdeco-entity-lockup__content ember-view']/div[2]/a)")

            # try for if there is any job with given title
            try:
                checkProduct=driver.find_element(By.XPATH,"//*[@class='t-24 t-black t-normal text-align-center']").text
            except:
                checkProduct=''

            ## if no job with given title found pass to next job title
            if(checkProduct=="No matching jobs found."):
                flag=0
            else:
                                                                                 # Specifying default value as ''
                for (i,job_link,comp_link) in itertools.zip_longest(info_div,job_Links,comp_Links, fillvalue=''):
                    data = []
                    try:
                        infos.append(i.text)   ## appending job info which contains (location,mode,job title,company name) separated by newline
                    except:
                        pass

                    try:
                         ## appending carrer title
                        Catogery.append(Title) 
                    except:
                        Catogery.append('')

                    try:
                        ## appendig state location
                        State.append(Location)
                    except:
                        State.append(Location)

                    try:
                        ## appending job links
                        Job_links.append(job_link.get_attribute('href'))
                    except:
                        Job_links.append('')

                    try:
                        comp_links.append(comp_link.get_attribute('href'))   ## appending company links in separate list to scrape their data later on
                        ## appending company links
                        company_links.append(comp_link.get_attribute('href'))
                    except:
                        company_links.append('')


                next_page_url = current_url + "&start=" + str(page_no)
                page_no=page_no+25
                
                try:
                    checkNextPageButton=driver.find_element(By.XPATH,"//*[@class='jobs-search-results-list__pagination pv5 artdeco-pagination ember-view']").text
                    if(checkNextPageButton):
                        driver.get(next_page_url) 
                except:
                    checkNextPageButton=''
                    flag=0

In [ ]:
## making location, mode , job title, company name from info above scraped due to they are separated by newline only in same locator
def make(info):
    lst = []
    string = ""
    for i in info:
        if i!="\n":
            string+=i
        if i=="\n": 
            lst.append(string)
            string=""
            
    lst.append(string)   
    return lst
    

In [ ]:
for info in infos:
    data=[]
    lst = make(info)
    try:
        ## appending job title
        Job_Title.append(lst[0])
    except:
        Job_Title.append('')
        
    try:
        ## appending company name
        Company_Name.append(lst[1])
    except:
        Company_Name.append('')
    
    try:
        ## appending job title 
        Job_location.append(lst[2])
    except:
        Job_location.append()
        
    try:
        ## appending mode of work
        Mode.append(lst[3])
    except:
        Mode.append('')
   

In [ ]:
def getNoemployee(employeesNo):
    for div in employeesNo:
        div = div.text
        last=div.partition(' ')[-1]
        if last=='employees':
            return div
              

In [ ]:
comp_links = set(comp_links)   ## getting set of company links so not to repeat same company in scraping process

In [ ]:
df_bike = pd.DataFrame(columns=['Company Name','Discription','employees','location'])
        
        ## according to above dataframe making excel file
df_bike.to_excel(f'company.xlsx',index=False)

In [ ]:
workbook_name = 'company.xlsx'          ### getting excel file
wb = load_workbook(workbook_name)      ### opening/loading excel excel file
page = wb.active                       ### running excel file to append retrive data in it 

In [ ]:
for comp in comp_links:  ## getting link by link company info from comp_links list made during above process
    data=[]
    url = comp + 'about/'  ## definning url with about section of company
        
    driver.get(url)
    
    ## company name locator
    try:
        company_name=driver.find_element(By.XPATH,"//*[@class='block mt2']") 
    except:
        company_name=''
        
    ## locator for company description
    try:
        description=driver.find_element(By.XPATH,"//*[@class='break-words white-space-pre-wrap mb5 text-body-small t-black--light']")
    except:
        description=''
    
    ## locator for numbers of employees 
    try:
        employeesNo=driver.find_elements(By.XPATH,"//*[@class='ember-view']/section/dl/dd")
    except:
        employeesNo=''
    
    ## finding proper numbers of empoyeesv work data
    try:
        employees = getNoemployee(employeesNo)
    except:
        employees=''
    
    ## locator for location of company
    try:
        try:
            location=driver.find_element(By.XPATH,"//*[@class='ember-view artdeco-card p0 mb4']//li//p")
        except:
            location=driver.find_element(By.XPATH,"//*[@class='ember-view artdeco-card p0 mb4']//p")
    except:
        location=''
    
    ## appending company name, description, number of employees, location
    try:
        data.append(company_name.text)
    except:
        data.append('')
    try:
        data.append(description.text)
    except:
        data.append('')
    try:
        data.append(employees)
    except:
        data.append('')
    try:
        data.append(location.text)
    except:
        data.append('')
    
    page.append(data)
    wb.save(filename=workbook_name) ## saving data to excel file made initially

In [ ]:
for i in range(len(Job_Title)):
    df_bike_ = df_bike_.append({'Catogery':Catogery[i],'State':State[i],'Job links':Job_links[i], 'company links':company_links[i],
                              'Job Title':Job_Title[i],'Job location':Job_location[i],
                              'Company Name':Company_Name[i], 'Mode':Mode[i]}, ignore_index=True)

df_bike_.to_excel('job.xlsx',index=False)

In [ ]:
driver.quit()  ## closing driver

In [ ]:
len(comp_links)